In [1]:
# %matplotlib notebook
# %matplotlib inline
import numpy as np
import pickle
np.random.seed(123)
import collections, copy, pickle
from importlib import reload
from dateutil.parser import parse
import scipy.linalg
import pandas as pd
import sklearn
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
from matplotlib import rcParams
rcParams['font.family'] = 'serif'
rcParams['font.size'] = 14
# rcParams['text.usetex'] = True
from IPython.display import HTML

In [2]:
from mlxtend.frequent_patterns import apriori

import sklearn.ensemble
import sklearn.svm
import sklearn.tree
import sklearn.linear_model
import sklearn.neighbors

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import sklearn.metrics

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [3]:
import util.plot
import util.string

In [4]:
# https://github.com/pbloem/machine-learning/blob/master/worksheets/Worksheet%202%2C%20Sklearn.ipynb

In [5]:
# data = pd.read_csv('ODI-2019-clean.csv', sep=';')
fn = 'ODI-2019-clean.pkl'
# load (old) data from disk
with open(fn, 'rb') as f:
    data = pickle.load(f)

In [6]:
# data.head()

## Categorical models

https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

# Discretization

https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features

https://scikit-learn.org/stable/auto_examples/preprocessing/plot_discretization_strategies.html#sphx-glr-auto-examples-preprocessing-plot-discretization-strategies-py

Strategies:
-    ‘uniform’: The discretization is uniform in each feature, which means that the bin widths are constant in each dimension.
-    quantile’: The discretization is done on the quantiled values, which means that each bin has approximately the same number of samples.
       - this causes outliers to be grouped together
-    ‘kmeans’: The discretization is based on the centroids of a KMeans clustering procedure.

In [7]:
class Encoders: pass
E = Encoders()
E.discretizers = {}
E.encoders = {}

In [8]:
key = 'Other'
# reload(util.data)
most_common = util.data.select_most_common(data.Programme, n=8, key=key)
value = np.array(list(most_common.values()))
# note that pd.where different than np.where
keys = most_common.keys()
data.Programme.where(data.Programme.isin(keys), key, inplace=True)

In [9]:
def discretize(data, k, n_bins=5):
    X = data[k]
    avg = np.nanmedian([x for x in X])
    X = np.where(np.isnan(X), avg, X)
    X = X.reshape(-1,1)
    bins = np.repeat(n_bins, X.shape[1]) # e.g. [5,3] for 2 features
    # encode to integers
    est = preprocessing.KBinsDiscretizer(n_bins=bins, encode='ordinal', strategy='kmeans')
    est.fit(X)
    data[k + ' bin'] = est.transform(X)
    E.discretizers[k] = est
    s = ''
    for st in [round(a,3) for a in est.bin_edges_[0]]:
        if k == 'Year':
            st = int(round(st))
        s += str(st) + ', '
    print('%s: $\\{%s\\}$\n' % (k,s[:-2]))

In [10]:
numerical = ['Year', 'Money', 'Neighbours', 'Stress level']
for k in numerical:
    discretize(data, k)

Year: $\{1971, 1981, 1991, 1994, 1996, 2001\}$

Money: $\{0.0, 13.77, 37.19, 64.316, 88.56, 100.0\}$

Neighbours: $\{0.0, 12.711, 28.4, 42.5, 64.25, 80.0\}$

Stress level: $\{0.0, 15.307, 38.186, 62.413, 86.179, 100.0\}$



In [12]:
def init_encoder(columns):
    E.encoders['x'] = preprocessing.OneHotEncoder()
    enc = E.encoders['x']
    enc.fit(columns)
    return enc.transform(columns)

categorical = ['ML', 'IR', 'Stat', 'DB', 'Gender', 'Chocolate', 'Stand Up', 'Programme']
y = 'ML'
categorical.remove(y)
keys = [k + ' bin' for k in numerical] + categorical
X_enc = init_encoder(data[keys])
E.encoders['x'].categories_

[array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array([0.0, 1.0, 2.0, 3.0, 4.0], dtype=object),
 array(['No', 'Unknown', 'Yes'], dtype=object),
 array(['No', 'Unknown', 'Yes'], dtype=object),
 array(['No', 'Unknown', 'Yes'], dtype=object),
 array(['female', 'male', 'unknown'], dtype=object),
 array(['Fat', 'Neither', 'Slim', 'Unknown'], dtype=object),
 array(['no', 'unknown', 'yes'], dtype=object),
 array(['AI', 'BA', 'BIO', 'CLS', 'CS', 'IS', 'Other', 'QRM'], dtype=object)]

In [13]:
def init_label_encoder(column):
    E.encoders['y'] = preprocessing.LabelEncoder()
    enc = E.encoders['y']
    enc.fit(column)
    return enc.transform(column)

Y_enc = init_label_encoder(data[y])
E.encoders['y'].classes_

array(['No', 'Yes'], dtype=object)

In [14]:
X_enc.shape, Y_enc.shape

((276, 47), (276,))

In [15]:
x_train, x_test, y_train, y_test = train_test_split(X_enc, Y_enc, test_size=0.5)
x_train.shape, y_train.shape

((138, 47), (138,))

In [31]:
def cross_validation(model_func, x_train, y_train, k=None, results=None, v=0):
    # Train for 5 folds, returing ROC AUC. You can also try 'accuracy' as a scorer
    n_folds = 5
    scores_acc = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring='accuracy') # roc_auc accuracy
    scores_roc = cross_val_score(model_func, x_train, y_train, cv=n_folds, scoring='roc_auc') # roc_auc accuracy
    if results is not None:
        results[k] = (scores_acc, scores_roc)
    if v:
        print('scores per fold ', scores_acc)
        print('  mean score    ', np.mean(scores_acc))
        print('  standard dev. ', np.std(scores_acc))

In [32]:
models = {
          'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear',
                                                           multi_class='ovr'),
#           'SGD': sklearn.linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3),
#           'SVC auto': sklearn.svm.SVC(gamma='auto'), 
          'SVC': sklearn.svm.SVC(kernel='linear'), 
#           'SVC polynomial': sklearn.svm.SVC(kernel='poly', gamma='auto', degree=4),    
          'Decision Tree':  sklearn.tree.DecisionTreeClassifier(),
          'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
#           'KNN 10': sklearn.neighbors.KNeighborsClassifier(n_neighbors=10),
          'Ensemble Random Forest': sklearn.ensemble.RandomForestClassifier(n_estimators=100),
#           'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=100)
         }

results = {}
for k,m in models.items():
    print(k)
    cross_validation(m, x_train, y_train, k, results)

Logit
SVC
Decision Tree
KNN 5
Ensemble Random Forest


In [35]:
print('Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\\\ \n\\hline')
best_k = ''
best_mean = 0
for k, (scores_acc, scores_roc) in results.items():
    if np.mean(scores_acc) > best_mean:
        best_mean = np.mean(scores_acc)
        best_k = k
    print('%s & %0.4f & %0.4f & %0.4f & %0.4f\\\\' % (k, np.mean(scores_acc), np.std(scores_acc), np.mean(scores_roc), np.std(scores_roc)))
print('\nbest acc:', best_k, round(best_mean,4))

Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\ 
\hline
Logit & 0.7098 & 0.0571 & 0.7031 & 0.1111\\
SVC & 0.7029 & 0.0729 & 0.6869 & 0.1148\\
Decision Tree & 0.5786 & 0.0865 & 0.5155 & 0.1119\\
KNN 5 & 0.7392 & 0.0417 & 0.6568 & 0.0591\\
Ensemble Random Forest & 0.7452 & 0.0765 & 0.6986 & 0.0969\\

best acc: Ensemble Random Forest 0.7452


In [36]:
print('Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\\\ \n\\hline')
best_k = ''
best_mean = 0
for k, (scores_acc, scores_roc) in results.items():
    if np.mean(scores_roc) > best_mean:
        best_mean = np.mean(scores_roc)
        best_k = k
print('\nbest roc:', best_k, round(best_mean,4))

Model & Mean Acc & Std Acc & Mean ROC & Std ROC \\ 
\hline

best roc: Logit 0.7031


In [20]:
# reinit models
models = {
          'Logit': sklearn.linear_model.LogisticRegression(solver='liblinear',
                                                           multi_class='ovr'),
#           'SGD': sklearn.linear_model.SGDClassifier(loss="hinge", penalty="l2", max_iter=1000, tol=1e-3),
#           'SVC auto': sklearn.svm.SVC(gamma='auto'), 
          'SVC': sklearn.svm.SVC(kernel='linear'), 
#           'SVC polynomial': sklearn.svm.SVC(kernel='poly', gamma='auto', degree=4),    
          'Decision Tree':  sklearn.tree.DecisionTreeClassifier(),
          'KNN 5': sklearn.neighbors.KNeighborsClassifier(n_neighbors=5),
#           'KNN 10': sklearn.neighbors.KNeighborsClassifier(n_neighbors=10),
          'Ensemble Random Forest': sklearn.ensemble.RandomForestClassifier(n_estimators=100),
#           'Ensemble Bagging': sklearn.ensemble.BaggingClassifier(n_estimators=100)
         }

# train best model on whole dataset
model = models[best_k]
model.fit(x_train, y_train)
y_pred = model.predict(x_test)

for v in [sklearn.metrics.accuracy_score(y_test, y_pred), 
          sklearn.metrics.roc_auc_score(y_test, y_pred)]:
    print(round(v,4))

0.6522
0.6288


In [52]:
best_k = 'Ensemble Random Forest'
model = models[best_k]
for k,v in results.items():
    if k != best_k:
        i = 0
        s,p = scipy.stats.ttest_ind(v[i], results[best_k][i])
        print(k,s,p, p < 0.05)

Logit -0.7425503772682113 0.4789872851462843 False
SVC -0.8011474640503641 0.44618595862459 False
Decision Tree -2.88614543293305 0.02031897748461718 True
KNN 5 -0.13963134782656522 0.8924025856403036 False


Both the acc and roc are not always significant

In [21]:
subkeys = []
for i,k in enumerate(keys):
    for v in E.encoders['x'].categories_[i]:
        subkeys.append(k + '_' + str(v))

assert len(subkeys) == pd.DataFrame(X_enc.toarray()).shape[1]

In [92]:
# model.fit(X_enc, Y_enc)
indices = np.argsort(model.feature_importances_)
indices = np.flip(indices)
n = 3
print('best features: indices, values')
indices[:n], model.feature_importances_[indices[:n]]

best features: indices, values


(array([22, 42, 26]), array([0.04854745, 0.04773113, 0.0440498 ]))

In [91]:
for i in indices[:3]:
    vec = np.zeros(X_enc.shape[1])
    vec[i] = 1
    print(subkeys[i])

IR_Yes
Programme_CLS
DB_No


# Association rules

In [22]:
# data_enc = pd.DataFrame(X_enc.toarray(), columns=subkeys, dtype=bool)
data_enc = pd.SparseDataFrame(X_enc, columns=subkeys, default_fill_value=False)
data_enc.head()

,Year bin_0.0,Year bin_1.0,Year bin_2.0,Year bin_3.0,Year bin_4.0,Money bin_0.0,Money bin_1.0,Money bin_2.0,Money bin_3.0,Money bin_4.0,...,Stand Up_unknown,Stand Up_yes,Programme_AI,Programme_BA,Programme_BIO,Programme_CLS,Programme_CS,Programme_IS,Programme_Other,Programme_QRM
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [23]:
# http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/
frequent_itemsets = apriori(data_enc, min_support=0.6, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets

,support,itemsets,length
0,0.967391,(Neighbours bin_0.0),1
1,0.876812,(Stat_Yes),1
2,0.659420,(Gender_male),1
3,0.902174,(Stand Up_no),1
4,0.855072,"(Stat_Yes, Neighbours bin_0.0)",2
5,0.637681,"(Gender_male, Neighbours bin_0.0)",2
6,0.880435,"(Stand Up_no, Neighbours bin_0.0)",2
7,0.804348,"(Stat_Yes, Stand Up_no)",2
8,0.789855,"(Stat_Yes, Neighbours bin_0.0, Stand Up_no)",3


In [24]:
frequent_itemsets[ (frequent_itemsets['length'] >= 3) &
                   (frequent_itemsets['support'] >= 0.6) ]

,support,itemsets,length
8,0.789855,"(Stat_Yes, Neighbours bin_0.0, Stand Up_no)",3


In [25]:
frequent_itemsets[ (frequent_itemsets['length'] >= 2) &
                   (frequent_itemsets['support'] >= 0.7) ]

,support,itemsets,length
4,0.855072,"(Stat_Yes, Neighbours bin_0.0)",2
6,0.880435,"(Stand Up_no, Neighbours bin_0.0)",2
7,0.804348,"(Stat_Yes, Stand Up_no)",2
8,0.789855,"(Stat_Yes, Neighbours bin_0.0, Stand Up_no)",3
